# Tiling Algorithm

Here is where I play with the tiling algorithm

In [1]:
%matplotlib inline

In [2]:
def get_redshift(cat):
    
    z = (u.Quantity(cat['cz']) / c.c).to(u.dimensionless_unscaled)
    MK = cat['Ktmag'] - cosmo.distmod(z)
    
    return z

In [3]:
import healpy as hp
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

from matplotlib import pyplot as plt
from astropy.cosmology import WMAP9 as cosmo
from astropy.table import Column
import astropy.units as u
import astropy.constants as c
from matplotlib import colors
import math as m
import numpy as np

from astroquery.skyview import SkyView
from astropy.coordinates import SkyCoord
from astroquery.irsa_dust import IrsaDust
from astropy.io import fits as pyfits
import glob
#!pip install aplpy
import aplpy

from astropy.utils.data import download_file

url = ('https://dcc.ligo.org/P1500071/public/18869_bayestar.fits.gz')
filename = download_file(url, cache=True)

from astroquery.vizier import Vizier

Vizier.ROW_LIMIT = -1
cat, = Vizier.get_catalogs('J/ApJS/199/26/table3')
len(cat)

z = get_redshift(cat)

#prob = hp.read_map(filename)

#prob, distmu, distsigma, distnorm = hp.read_map(filename, field=[0, 1, 2, 3])

prob, distmu, distsigma, distnorm = hp.read_map(filename, field=range(4))

npix = len(prob)
print(npix)
nside = hp.npix2nside(npix)
pixarea = hp.nside2pixarea(nside)
pixarea

/student/student_i4/miniconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
/student/student_i4/miniconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2287: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)
/student/student_i4/miniconda3/lib/python3.5/site-packages/astropy/cosmology/core.py:1414: RuntimeWarning: divide by zero encountered in log10
  val = 5. * np.log10(abs(self.luminosity_distance(z).value)) + 25.0


NSIDE = 512
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
Ordering converted to RING
3145728


3.994741635118857e-06

In [4]:
cat_now = cat
print(type(cat))

<class 'astropy.table.table.Table'>


In [5]:
r = cosmo.luminosity_distance(z).to('Mpc').value
theta = 0.5*np.pi - cat_now['_DEJ2000'].to('rad').value
phi = cat_now['_RAJ2000'].to('rad').value
ipix = hp.ang2pix(nside, theta, phi)

/student/student_i4/miniconda3/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)
/student/student_i4/miniconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:2287: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [6]:
dp_dV = prob[ipix]*distnorm[ipix]*norm(distmu[ipix], distsigma[ipix]).pdf(r) / pixarea

/student/student_i4/miniconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in greater_equal
  return (self.a <= x) & (x <= self.b)
/student/student_i4/miniconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:872: RuntimeWarning: invalid value encountered in less_equal
  return (self.a <= x) & (x <= self.b)


In [7]:
cands = cat_now[dp_dV >= np.nanpercentile(dp_dV,95)]

/student/student_i4/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in greater_equal
  if __name__ == '__main__':


In [8]:
print(len(cands))

2177


In [28]:
cands[20:]

_RAJ2000,_DEJ2000,ID,A,RAJ2000,DEJ2000,GLON,GLAT,Kcmag,e_Kcmag,Hcmag,e_Hcmag,Jcmag,e_Jcmag,Ktmag,e_Ktmag,Htmag,e_Htmag,Jtmag,e_Jtmag,E_B-V_,Riso,Rext,b_a,flags,type,r_type,cz,e_cz,n_cz,r_cz,CAT,_2MX,SimbadName,NED
deg,deg,,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,[arcsec],[arcsec],,,,,km / s,km / s,,,,,,
float64,float64,bytes16,bytes1,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes4,bytes5,bytes2,int32,int16,bytes1,bytes19,bytes28,bytes3,bytes26,bytes3
344.30655,-43.39614,22571357-4323461,,344.30655,-43.39614,350.23697,-61.80075,7.604,0.018,7.839,0.016,8.473,0.016,7.450,0.034,7.672,0.026,8.279,0.022,0.012,1.903,2.231,0.810,Z000,-2A_T,ZC,1689,45,6,20096dF...C...0000J,g2257135-432346,2MX,2MASX J22571357-4323461,NED
349.04486,-42.58483,23161076-4235053,,349.04486,-42.58483,348.14471,-65.23586,7.620,0.016,7.936,0.016,8.606,0.015,7.531,0.020,7.832,0.017,8.481,0.017,0.014,1.883,2.122,0.590,Z000,2B_P,ZC,1600,45,6,20096dF...C...0000J,g2316107-423505,2MX,2MASX J23161076-4235053,NED
47.43964,-20.57930,03094551-2034454,,47.43964,-20.57930,208.77309,-57.80897,7.720,0.023,8.038,0.020,8.532,0.017,7.372,0.047,7.768,0.039,8.160,0.025,0.026,1.983,2.279,1.000,Z133,5X1T,ZC,1651,45,6,20096dF...C...0000J,g0309454-203445,2MX,2MASX J03094551-2034454,NED
46.03325,-26.06963,03040796-2604105,,46.03325,-26.06963,218.54221,-60.40433,7.734,0.019,7.950,0.012,8.631,0.009,7.671,0.019,7.875,0.012,8.536,0.009,0.016,1.943,2.081,0.560,0000,-2A_R,ZC,1649,45,6,20096dF...C...0000J,g0304080-260411,2MX,2MASX J03040796-2604105,NED
53.75563,-24.93311,03350135-2455592,,53.75563,-24.93311,218.95412,-53.34695,7.828,0.021,8.063,0.018,8.724,0.017,7.621,0.039,7.899,0.029,8.538,0.023,0.026,1.947,2.231,0.720,Z333,1X_T,ZC,1405,45,6,20096dF...C...0000J,g0335013-245560,2MX,2MASX J03350135-2455591,NED
54.87851,-18.68817,03393085-1841172,,54.87851,-18.68817,209.70656,-50.57322,7.858,0.017,8.076,0.011,8.767,0.010,7.788,0.018,8.001,0.011,8.690,0.010,0.065,1.767,1.917,0.880,0555,-3A,ZC,530,45,6,20096dF...C...0000J,g0339308-184117,2MX,2MASX J03393085-1841172,NED
49.92116,-19.41136,03194107-1924408,,49.92116,-19.41136,208.16510,-55.22260,7.885,0.025,8.091,0.021,8.812,0.018,7.553,0.047,7.752,0.036,8.475,0.027,0.030,2.120,2.392,0.460,Z111,4B1T,ZC,1620,45,6,20096dF...C...0000J,g0319411-192440,2MX,2MASX J03194107-1924408,NED
189.35431,74.19189,12372503+7411307,,189.35431,74.19189,124.23476,42.90103,7.905,0.017,8.125,0.016,8.832,0.016,7.748,0.023,7.955,0.021,8.679,0.019,0.028,1.842,2.095,0.810,Z133,-5,ZC,1980,14,N,1991RC3.9.C...0000d,NGC_4589,2MX,2MASX J12372503+7411307,NED


In [9]:
tiles = []

In [43]:
def get_tiles(cands, tiles):
    tile_cands = []
    max_score = []
    new_cands = cands
    k = 0
    for cand in cands:
        if k%100 == 0:
            print('indexing... ' + str(k))

        ra = cand['_RAJ2000']
        dec = cand['_DEJ2000']
        cords = SkyCoord(ra_tr, dec_tr, 'fk5', unit='deg')

        # Top Left
        top_left = {}
        top_left['name'] = 'Top Left on ' + str(cand['SimbadName'])
        top_left['ra_min'] = ra - 0.05*np.cos(coords.ra.to('radian'))
        top_left['ra_max'] = ra + 2.55*np.cos(coords.ra.to('radian'))
        top_left['dec_max'] = dec + np.float64(0.05)
        top_left['dec_min'] = dec - np.float64(1.95)
        top_left['center_ra'] = 0.5*(top_left['ra_min'] + top_left['ra_max'])
        top_left['center_dec'] = 0.5*(top_left['dec_min'] + top_left['dec_max'])

        # Top Right
        top_right = {}
        top_right['name'] = 'Top Right on ' + str(cand['SimbadName'])
        top_right['ra_max'] = ra + 0.05*np.cos(coords.ra.to('radian'))
        top_right['ra_min'] = ra - 2.55*np.cos(coords.ra.to('radian'))
        top_right['dec_max'] = dec + 0.05
        top_right['dec_min'] = dec - 1.95
        top_right['center_ra'] = 0.5*(top_right['ra_min'] + top_right['ra_max'])
        top_right['center_dec'] = 0.5*(top_right['dec_min'] + top_right['dec_max'])

        # Bottom Left
        bottom_left = {}
        bottom_left['name'] = 'Bottom Left on ' + str(cand['SimbadName'])
        bottom_left['ra_min'] = ra - 0.05*np.cos(coords.ra.to('radian'))
        bottom_left['ra_max'] = ra + 2.55*np.cos(coords.ra.to('radian'))
        bottom_left['dec_min'] = dec - 0.05
        bottom_left['dec_max'] = dec + 1.95
        bottom_left['center_ra'] = 0.5*(bottom_left['ra_min'] + bottom_left['ra_max'])
        bottom_left['center_dec'] = 0.5*(bottom_left['dec_min'] + bottom_left['dec_max'])

        # Bottom Right
        bottom_right = {}
        bottom_right['name'] = 'Bottom Right on ' + str(cand['SimbadName'])
        bottom_right['ra_max'] = ra + 0.05*np.cos(coords.ra.to('radian'))
        bottom_right['ra_min'] = ra - 2.55*np.cos(coords.ra.to('radian'))
        bottom_right['dec_min'] = dec - 0.05
        bottom_right['dec_max'] = dec + 1.95
        bottom_right['center_ra'] = 0.5*(bottom_right['ra_min'] + bottom_right['ra_max'])
        bottom_right['center_dec'] = 0.5*(bottom_right['dec_min'] + bottom_right['dec_max'])

        # Centered
        centered = {}
        centered['name'] = 'Centered on ' + str(cand['SimbadName'])
        centered['ra_min'] = ra - 1.3*np.cos(coords.ra.to('radian'))
        centered['ra_max'] = ra + 1.3*np.cos(coords.ra.to('radian'))
        centered['dec_max'] = dec + 1.0
        centered['dec_min'] = dec - 1.0
        centered['center_ra'] = ra
        centered['center_dec'] = dec

        cords = [top_left,top_right,bottom_left,bottom_right,centered]

        for cord in cords:
            tile = {}
            tile['gal_indexes'] = []
            center_coords = SkyCoord(float(cord['center_ra']), float(cord['center_dec']), 'fk5', unit='deg')

            tile['properties'] = {'name': cord['name'],
                                  'coords': center_coords.to_string('hmsdms'),
                                  'coord_ra': cord['center_ra'],
                                  'coord_dec': cord['center_dec'],
                                  'score': 0}

            keep = (cands['_RAJ2000']<=cord['ra_max']) & (cands['_RAJ2000']>=cord['ra_min']) & (cands['_DEJ2000']<=cord['dec_max']) & (cands['_DEJ2000']>=cord['dec_min'])

            cands2 = cands[keep]

            tile['galaxies'] = []

            for cand_k in cands2:

                cand_coords = SkyCoord(float(cand_k['_RAJ2000']), float(cand_k['_DEJ2000']), 'fk5', unit='deg')

                tile['galaxies'].append({'name': cand_k['SimbadName'],
                                         'coords': cand_coords.to_string('hmsdms'),
                                         'anything else': 'other stuff'})

                tile['gal_indexes'].append(k)

            tile['properties']['score'] = len(tile['galaxies'])

            if tile['properties']['score'] >= max(max_score):
                tile_cands.append(tile)
                max_score.append({'score': tile['properties']['score'], 'coords': [tile['coord_ra'], tile['coord_dec']]})
                print('New max score: ', max(max_score['score']))

        k = k+1

    for j in range(len(tile_cands)):
        tile = tile_cands[j]
        if tile['properties']['score'] == max_score:

            tiles.append(tile)
            galaxy_indexes = np.array([tile['gal_indexes']])

            new_cands.remove_row(gal['index_in_catalog'])

    return new_cands

In [44]:
some_cands = cands
some_cands.remove_row(["b'2MASX J00545347-3741037'"])
print(len(some_cands))

TypeError: Row index must be an integer

In [45]:
def tile_sky(cands):
    tiles = []
    loop_cands = cands
    
    while len(loop_cands) > 0:
        print(str(len(loop_cands))+' candidates left')
        loop_cands = get_tiles(loop_cands, tiles)
    return tiles

In [46]:
tiles = tile_sky(cands)

2177 candidates left
indexing... 0
New max score:  5


New max score:  7
New max score:  7
New max score:  13
New max score:  16
New max score:  16
New max score:  16
New max score:  17
New max score:  20
indexing... 100
New max score:  21
New max score:  21
New max score:  24
indexing... 200
New max score:  26
indexing... 300
New max score:  29
indexing... 400
indexing... 500
indexing... 600
indexing... 700
indexing... 800
indexing... 900
indexing... 1000
indexing... 1100
indexing... 1200
indexing... 1300
indexing... 1400
indexing... 1500
indexing... 1600
indexing... 1700
indexing... 1800
indexing... 1900
indexing... 2000
indexing... 2100
2148 candidates left
indexing... 0
New max score:  5
New max score:  7
New max score:  7
New max score:  12
New max score:  16
New max score:  16
New max score:  17
New max score:  20
indexing... 100
New max score:  21
New max score:  21
New max score:  24
indexing... 200
New max score:  25
indexing... 300
New max score:  25
indexing... 400
indexing... 500
New max score:  25
indexing... 600
indexing... 7

IndexError: index 2017 is out of bounds for axis 0 with size 1929